In [60]:
import datetime, os, time, pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import losses, models, layers, optimizers, callbacks, utils
from tensorflow.keras.layers.experimental import preprocessing

In [61]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [62]:
data_dir = pathlib.Path("D:Dataset/랜드마크 이미지/Validation/")

In [63]:
image_count = len(list(data_dir.glob('*/*.jpg'))) # 총 이미지 개수
test_ = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)

In [64]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [65]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [66]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [77]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [78]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))

In [79]:
len(a)

1504

In [80]:
arg = np.argmax(a,axis=1)

In [93]:
for i, j in zip(a[:5], arg[:5]):
    print(i[j])

1.0
1.0
1.0
1.0
1.0


In [95]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE
INPUT_SIZE = [224, 240, 260, 300, 380]
result = pd.DataFrame([])
result_conf = pd.DataFrame([])

for size in [224, 240, 260, 300, 380]:
    SIZE = size
    img_height = SIZE
    img_width = SIZE
    image_size=(img_height, img_width)
    test_ds = test_.map(process_path, num_parallel_calls=AUTOTUNE)
    test_ds = configure_for_performance(test_ds)
    model = tf.keras.models.load_model(f"model_{size}_2.h5")
    a = model.predict(test_ds)
    arg = np.argmax(a,axis=1)
    conf = []
    for i, j in zip(a, arg):
        conf.append(i[j])
    result[f"size_{size}"] = arg
    result_conf[f"size_{size}_conf"] = conf

In [22]:
result.mode(axis=1)[0]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
1499    83.0
1500    83.0
1501    83.0
1502    83.0
1503    83.0
Name: 0, Length: 1504, dtype: float64

In [98]:
result["answer"] = result.mode(axis=1)[0]
result["answer"] = result["answer"].astype("int")

In [100]:
result

,size_224,size_240,size_260,size_300,size_380,answer
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
1499,83,83,83,83,83,83
1500,83,83,83,83,83,83
1501,83,83,83,83,83,83
1502,83,83,83,83,83,83


In [101]:
test_ds = test_.map(process_path, num_parallel_calls=AUTOTUNE)
temp = []
for i, j in test_ds:
    temp.append(j.numpy())
result["Real"] = temp

In [102]:
result

,size_224,size_240,size_260,size_300,size_380,answer,Real
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1499,83,83,83,83,83,83,83
1500,83,83,83,83,83,83,83
1501,83,83,83,83,83,83,83
1502,83,83,83,83,83,83,83


In [104]:
result_conf["answer"] = result_conf.mean(axis=1)

In [106]:
def GAP_vector(pred, conf, true, return_x=False):
    x = pd.DataFrame({'pred': pred, 'conf': conf, 'true': true})
    x.sort_values('conf', ascending=False, inplace=True, na_position='last')
    x['correct'] = (x.true == x.pred).astype(int)
    x['prec_k'] = x.correct.cumsum() / (np.arange(len(x)) + 1)
    x['term'] = x.prec_k * x.correct
    gap = x.term.sum() / x.true.count()
    if return_x:
        return gap, x
    else:
        return gap

In [107]:
gap , x = GAP_vector(result["answer"], result_conf["answer"], result["Real"], return_x=True)

In [108]:
gap

0.9784071713571678

In [109]:
x

,pred,conf,true,correct,prec_k,term
0,0,1.000000,0,1,1.000000,1.000000
909,50,1.000000,50,1,1.000000,1.000000
928,51,1.000000,51,1,1.000000,1.000000
927,51,1.000000,51,1,1.000000,1.000000
926,51,1.000000,51,1,1.000000,1.000000
...,...,...,...,...,...,...
1250,67,0.802079,67,1,0.980667,0.980667
267,14,0.799807,14,1,0.980680,0.980680
270,18,0.785963,15,0,0.980027,0.000000
846,8,0.777344,46,0,0.979375,0.000000
